In [44]:
from dotenv import load_dotenv

load_dotenv()

import aisuite as ai

# Define the client. You can use this variable inside the functions!
CLIENT = ai.Client()


import PyPDF2
import re
import json

import sys
import io
import pandas as pd
from datetime import datetime
import json

In [45]:
# pip install rich
from rich.console import Console
from rich.markdown import Markdown

console = Console()

def pretty_print_markdown(text: str, title: str = "Visualization Verdict"):
    """Render Markdown-style LLM output nicely in the terminal."""
    console.rule(f"[bold cyan]{title}")
    md = Markdown(text)
    console.print(md)
    console.rule()


In [46]:
# Agent1: CSV Analyzer

def csv_analyzer(df, model: str = "anthropic:claude-haiku-4-5") -> str: 
    
    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f'''
    
    You are give a sample of a dataset of tweets.

    {df}
    
    Your task is to identify the row name of each row and what kind of data it is and what patter it is.

    Eg:
    Row Name: tweet_id, DataType: Integer, Pattern: 18... (19 digits)
    Row Name: created_at, DataType: DateTime, Pattern: YYYY-MM-DDTHH:MM:SS

    Like this I want for all the rows in the dataset.
    Give me only this in a json array format and no additional text.
    
    ''' 

    ### END CODE HERE ###
    
    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[

            {"role": "system", "content": "You are a well experienced data analyst."},
            {"role": "user", "content": prompt}
            ],
        temperature=1.0,
    )

    return response.choices[0].message.content

In [47]:
df = pd.read_csv("tweets.csv")


In [48]:
import pandas as pd



# Convert to JSON string in 'records' format
json_string = df.to_json(orient='records')

# Write to a new JS file as a variable assignment
with open('output.js', 'w') as f:
    f.write("const jsonData = " + json_string + ";\n")


In [49]:
# df.to_json('output.json', orient='records', indent=4)

In [50]:
csv_analysis = csv_analyzer(df.head(10))

In [51]:
pretty_print_markdown(csv_analysis, "CSV Analysis")

────────────────────────────────────────────────── CSV Analysis ───────────────────────────────────────────────────

                                                                                                                   
 [                                                                                                                 
   {                                                                                                               
     "row_name": "tweet_id",                                                                                       
     "data_type": "Integer",                                                                                       
     "pattern": "19 digits starting with 189"                                                                      
   },                                                                                                              
   {                                                                                                               
     "row_name": "conversation_id",                                                                                
     "data_type": "Integer",                                                                                       
     "pattern": "19 digits starting with 189"                                                                      
   },                                                                                                              
   {                                                                                                               
     "row_name": "created_at",                                                                                     
     "data_type": "DateTime",                                                                                      
     "pattern": "YYYY-MM-DDTHH:MM:SS"                                                                              
   },                                                                                                              
   {                                                                                                               
     "row_name": "user_id",                                                                                        
     "data_type": "Integer",                                                                                       
     "pattern": "8 digits starting with 10000"                                                                     
   },                                                                                                              
   {                                                                                                               
     "row_name": "username",                                                                                       
     "data_type": "String",                                                                                        
     "pattern": "Alphanumeric lowercase, 8-12 characters"                                                          
   },                                                                                                              
   {                                                                                                               
     "row_name": "display_name",                                                                                   
     "data_type": "String",                                                                                        
     "pattern": "Mixed case text with initial capitals and optional surname initial"                               
   },                                                                                                              
   {                                                                                                               
     "row_name": "text",                                                                                           
     "data_type": "String",                             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [52]:
# Agent2: Visualiztion Decision

def viz_decider(csv_analysis, model: str = "anthropic:claude-haiku-4-5") -> str: 
    
    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f'''
    
    Based ont the description of tweets dataset: {csv_analysis}

    Youe taksk is to suggest the top 3 visualizations that would best represent the data.

    Name of the Plot - Categorical / Numerical - Reason for choosing this plot.

    Give me the top 3 visualization in this format without any additonal text.
    
    ''' 

    ### END CODE HERE ###
    
    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[

            {"role": "system", "content": "You are a well experienced data analyst."},
            {"role": "user", "content": prompt}
            ],
        temperature=1.0,
    )

    return response.choices[0].message.content

In [53]:
viz_verdict = viz_decider(csv_analysis)

In [54]:
pretty_print_markdown(viz_verdict)

────────────────────────────────────────────── Visualization Verdict ──────────────────────────────────────────────

 1 Time Series Line Chart - Numerical - To track tweet volume, engagement metrics (likes, retweets, replies), and  
   trends over time using the created_at timestamp, revealing patterns in user activity and content performance.   
 2 Bar Chart - Categorical - To compare engagement metrics (like_count, retweet_count, reply_count) across         
   languages (lang), sources, or top hashtags, identifying which categories drive the most interaction.            
 3 Scatter Plot - Numerical - To analyze the relationship between multiple engagement metrics (e.g., like_count vs 
   retweet_count, or tweet length vs reply_count), revealing correlation patterns and outlier tweets with          
   exceptional performance.

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [55]:
# Agent2.5: Visualiztion Planner

def viz_coder(viz_verdict, csv_analysis, model: str = "openai:gpt-4o-mini") -> str: 
    
    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f'''
    
    Based on the description of the visualization suggestions: {viz_verdict}

    The schema of the dataset is here: {csv_analysis}

    Your task is to plan how to generate me a html & js code 
    to generate the top most visualization from the visualization suggestions.

    The data set has been stored in a js file named 'output.js' 
    in the same folder where this generated html file be saved.

    Give me only the complete plan without any additional text.
    
    ''' 

    ### END CODE HERE ###
    
    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[

            {"role": "system", "content": "You are a well experienced data analyst and front end developer."},
            {"role": "user", "content": prompt}
            ],
        temperature=1.0,
    )

    return response.choices[0].message.content

In [56]:
html_plan = viz_coder(viz_verdict,csv_analysis)

In [57]:
pretty_print_markdown(html_plan)

────────────────────────────────────────────── Visualization Verdict ──────────────────────────────────────────────

  1 HTML Structure:                                                                                                
     • Create a basic HTML document structure (, , , ).                                                            
     • Include a title for the visualization.                                                                      
     • Create a  element with a unique ID (e.g., "time-series-chart") to hold the Time Series Line Chart.          
  2 Include Libraries:                                                                                             
     • Include a library for data visualization (e.g., Chart.js or D3.js) via a CDN in the  section.               
     • Include the 'output.js' file in the  section to access the dataset.                                         
  3 JavaScript Functions:                                                                                          
     • Create a script tag in the  to write JavaScript code.                                                       
     • Load the dataset from 'output.js' and ensure it's accessible as a variable (e.g., const data = outputData;).
     • Parse the created_at field to extract timestamp data and convert it into a format suitable for the charting 
       library.                                                                                                    
     • Collect engagement metrics (likes, retweets, replies) from the dataset and prepare them for plotting.       
  4 Time Series Data Preparation:                                                                                  
     • Create a mapping function to group the data by time intervals (e.g., by day, week, or month) and calculate  
       aggregate metrics (total likes, retweets, and replies) for each interval.                                   
  5 Chart Configuration:                                                                                           
     • Define the chart configuration object, including:                                                           
        • Type: 'line'.                                                                                            
        • Data: Set up labels (time intervals) and datasets (likes, retweets, replies).                            
        • Options: Configure responsive layout, axes titles, legends, colors, gridlines, and tooltips.             
  6 Render the Chart:                                                                                              
     • Use the charting library's method to render the line chart in the specified  based on the configuration.    
  7 Styling:                                                                                                       
     • Include some CSS styles in a  tag within the  to improve the visualization's appearance (e.g., set sizes,   
       margins, and background).                                                                                   
  8 Testing:                                                                                                       
     • Check the console for any errors in data loading or chart rendering to ensure everything works as expected. 
     • Manually verify the output for correctness.                                                                 
  9 Final Touches:                                                                                                 
     • Add any interactivity features (like tooltips or clickable elements) as extra enhancements if needed.       
     • Ensure the visual is mobile responsive, testing it on different screen sizes.                               
 10 Save and Document:                                                                                             
     • Save the final HTML file, ensuring all references are correct.                                              
     • Add comments in the JavaScript code for clarity a

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [58]:
# Agent3: Visualiztion Code

def viz_coder(html_plan, csv_analysis, model: str = "anthropic:claude-haiku-4-5") -> str: 
    
    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f'''
    
    Based on the description of the visualization plan: {html_plan}

    The schema of the dataset is here: {csv_analysis}

    Your task is to generate me a html & js code 
    to generate the top most visualization from the visualization suggestions.

    The data set has been stored in a js file named 'output.js' 
    in the same folder where this generated html file be saved.

    Give me only the complete html code without any additional text.
    
    ''' 

    ### END CODE HERE ###
    
    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[

            {"role": "system", "content": "You are a well experienced data analyst and front end developer."},
            {"role": "user", "content": prompt}
            ],
        temperature=1.0,
    )

    return response.choices[0].message.content

In [59]:
html_code = viz_coder(html_plan,csv_analysis)

In [ ]:
pretty_print_markdown(html_code, "Visualization")

In [60]:
import re
from pathlib import Path

def save_fenced_html(s: str, out_path: str | Path) -> Path:
    """
    Extracts HTML from a fenced block like:
    ```html
    ...html here...
    ```
    and saves it to out_path.
    If no fence is found, saves s as-is.
    """
    # Match ```html ... ``` OR ```HTML ... ```
    m = re.search(r"^```html\s*(.*?)\s*```$", s, flags=re.DOTALL | re.IGNORECASE)
    html = m.group(1) if m else s
    out_path = Path(out_path)
    out_path.write_text(html, encoding="utf-8")
    return out_path



save_fenced_html(html_code, "viz.html")
print("Saved to viz.html")

Saved to viz.html


In [61]:
# Agent4: Evaluate Code

def eval_code(html_plan,html_code, model: str = "anthropic:claude-haiku-4-5") -> str: 
    
    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f'''
    
    You have the entire HTML code: {html_code}

    This was the plan: {html_plan}

    Evaluate the code for it's effectiveness in visualizing code.

    Check if the "jsonData" variable from ouput.js is being used properly.

    Note: This html code will be saved in the future in the same folder as the output.js file.
    So make sure the code is using the output.js file properly.

    After making all these considerations, give me the revised html code.

    Only the html code without any additional text.
    
    ''' 

    ### END CODE HERE ###
    
    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[

            {"role": "system", "content": "You are a well experienced data analyst and front end developer."},
            {"role": "user", "content": prompt}
            ],
        temperature=1.0,
    )

    return response.choices[0].message.content

In [63]:
html_code2 = eval_code(html_plan,html_code)

In [64]:
save_fenced_html(html_code2, "viz2.html")
print("Saved to viz2.html")

Saved to viz2.html


In [65]:
# Agent5: Evaluate Code Again

def eval_code_again(html_plan,html_code, model: str = "openai:gpt-4o-mini") -> str: 
    
    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f'''
    
    You have the entire HTML code: {html_code}

    This was the plan: {html_plan}

    Evaluate the code for it's effectiveness in visualizing code.

    Check if the "jsonData" variable from ouput.js is being used properly.

    Note: This html code will be saved in the future in the same folder as the output.js file.
    So make sure the code is using the output.js file properly.

    Check if the code is complete.

    Check for the best practices in coding.

    If the logic is right.

    After making all these considerations, give me the revised html code.

    Only the entire revised html code without any additional text.
    
    ''' 

    ### END CODE HERE ###
    
    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[

            {"role": "system", "content": "You are a well experienced data analyst and front end developer."},
            {"role": "user", "content": prompt}
            ],
        temperature=1.0,
    )

    return response.choices[0].message.content

In [66]:
html_code3 = eval_code_again(html_plan,html_code2)

In [67]:
save_fenced_html(html_code3, "viz3.html")
print("Saved to viz3.html")

Saved to viz3.html


In [68]:
# Agent6: Rubric Anal

def rubric_analysis(html_code, model: str = "openai:gpt-4o-mini") -> str: 
    
    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f'''
    
    You have the entire HTML code: {html_code}

    Evaluate the code for the following metrics:

    - code completeness: 10
    - code correctness: 20
    - logic: 30
    - best practices: 10
    - parsing the jsonData correctly from the output.js: 30
    - colour co-ordination: 20
    - visual appeal: 10

    Based on this give me the score out of 130. and a score for all these metrics. No additional text.
    
    ''' 

    ### END CODE HERE ###
    
    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[

            {"role": "system", "content": "You are a well experienced data analyst and front end developer."},
            {"role": "user", "content": prompt}
            ],
        temperature=1.0,
    )

    return response.choices[0].message.content

In [69]:
rubric_anal = rubric_analysis(html_code3)

In [70]:
pretty_print_markdown(rubric_anal,"Rubric Analysis")

───────────────────────────────────────────────── Rubric Analysis ─────────────────────────────────────────────────

 • Code Completeness: 10/10                                                                                        
 • Code Correctness: 20/20                                                                                         
 • Logic: 30/30                                                                                                    
 • Best Practices: 10/10                                                                                           
 • Parsing the jsonData correctly from output.js: 25/30                                                            
 • Colour Co-ordination: 20/20                                                                                     
 • Visual Appeal: 10/10                                                                                            

Total Score: 125/130

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [71]:
rubric_anal = rubric_analysis(html_code)

In [72]:
pretty_print_markdown(rubric_anal,"Rubric Analysis Old")

─────────────────────────────────────────────── Rubric Analysis Old ───────────────────────────────────────────────

 • code completeness: 8                                                                                            
 • code correctness: 18                                                                                            
 • logic: 25                                                                                                       
 • best practices: 8                                                                                               
 • parsing the jsonData correctly from the output.js: 25                                                           
 • colour co-ordination: 20                                                                                        
 • visual appeal: 9                                                                                                

Total Score: 113 out of 130

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────